In [15]:
import http.client
import json
import pandas as pd
import requests
from datetime import datetime
from lxml import etree
from json.decoder import JSONDecodeError

import logging


In [11]:
logging.basicConfig(filename='log.log', filemode='a', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

init_doi = '10.1016/j.cosrev.2017.10.002'
edges_array = []
nodes_array = []
total_citations = 0
new_citation_lists = []

def get_json(article_doi):
    global total_citations
    total_citations += 1
    print(total_citations)
    url = 'https://opencitations.net/index/api/v1/metadata/{}'.format(article_doi)
    header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:32.0) Gecko/20100101 Firefox/32.0',}
    r = requests.get(url, headers=header)
    json_data = r.json()
    return json_data

def load_article(article_doi):
    json_data = get_json(article_doi)

    def build_node_list(article_data):
        nodes_array.append(article_data)

    def build_edge_list(citation_list):
        for i in range(len(citation_list)):
            edges_array.append([total_citations, citation_list[i].strip(), article_doi, 'citation', 1])
            json_data = get_json(citation_list[i].strip())
            article_data, citation_list_two = parse_json(json_data)
            build_node_list(article_data)

    article_data, citation_list = parse_json(json_data)

    build_node_list(article_data)
    build_edge_list(citation_list)

    df1 = pd.DataFrame(nodes_array, columns=['~id', '~label', 'oa_link', 'citation_count', 'source_title', 'year', 'author', 'title'])
    df1.to_csv('nodes.csv', index=None) # save df to file
    df2 = pd.DataFrame(edges_array, columns=['~id', '~from', '~to', '~label', 'strength'])
    df2.to_csv('edges.csv', index=None) # save df to file
    return citation_list

def parse_json(json_data):
    doi = json_data[0]['doi']
    oa_link = json_data[0]['oa_link']
    citation_count = json_data[0]['citation_count']
    citation_list = json_data[0]['citation'].split("; ")
    source_title = json_data[0]['source_title']
    year = json_data[0]['year']
    author = json_data[0]['author']
    title = json_data[0]['title']
    data_list = [doi, 'article', oa_link, citation_count, source_title, year, author, title]
    return data_list, citation_list


def reload(citation_list):
    y = citation_list
    try:
        new_citation_lists.remove(citation_list)
    except ValueError as ve:
        pass

    for citation in y:
        try:
            print(citation.strip())
            x = load_article(citation.strip())
            print(x)
            new_citation_lists.append(x)
        except JSONDecodeError as e:
            pass

    for citation_list in new_citation_lists:
#         print(citation_list)
        reload(citation_list)



In [12]:
citation_list = load_article(init_doi)
reload(citation_list)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
10.1145/3196398.3196444
58
59
['10.1145/3239235.3267434']
10.1007/978-3-030-03493-1_59
60
61
10.1177/0165551519877049
62
63
['10.1177/0165551520911590']
10.23919/fruct.2019.8711900
64
65
10.1007/s10803-019-04242-3
66
67
10.1177/2053951718823815
68
69
70
71
72
73
['10.3390/su11071961', '10.26565/2227-6521-2019-42-06', '10.30853/manuscript.2020.4.24', '10.1007/978-3-030-44999-5_39', '10.1177/2053951720933286']
10.4018/978-1-5225-8575-6.ch007
74
75
['10.4018/978-1-7998-2874-7.ch005']
10.1007/978-3-030-20055-8_49
76
77
10.1007/978-3-030-23943-5_16
78
79
10.1007/978-3-030-26061-3_28
80
81
10.1007/978-3-030-29513-4_22
82
83
10.1007/978-3-030-30577-2_21
84
85
10.1007/978-3-030-17065-3_10
86
87
10.1080/01587919.2019.1632171
88
89
10.1080/09720510.2018.1471263
90
91
['10.1080/02522667.2019.1616911']
10.1007/978-3-319-99007-1_32
92
93
1

In [32]:
import pandas as pd
import rdfpandas
import numpy as np
import rdflib
import rdflib.compare
df_new = pd.DataFrame([[10001,10001,10001,10002,10002],[1000,1001,1002,1001,1003]],).T
g = rdflib.Graph()    
for (index, series) in df_new.iterrows():
    for (column, value) in series.iteritems():
         g.add((rdflib.URIRef(index),rdflib.URIRef(column),rdflib.Literal(value)))

TypeError: 'int' object is not iterable

In [43]:
# create a Graph
g = rdflib.Graph()

# parse in an RDF file hosted on the Internet
result = g.parse("http://www.w3.org/People/Berners-Lee/card")

# loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")

# print the number of "triples" in the Graph
print("graph has {} statements.".format(len(g)))
# prints graph has 86 statements.

# print out the entire Graph in the RDF Turtle format
print(g.serialize(format="turtle").decode("utf-8"))

graph has 86 statements.
@prefix : <http://xmlns.com/foaf/0.1/> .
@prefix cc: <http://creativecommons.org/ns#> .
@prefix cert: <http://www.w3.org/ns/auth/cert#> .
@prefix con: <http://www.w3.org/2000/10/swap/pim/contact#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix doap: <http://usefulinc.com/ns/doap#> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sioc: <http://rdfs.org/sioc/ns#> .
@prefix solid: <http://www.w3.org/ns/solid/terms#> .
@prefix space: <http://www.w3.org/ns/pim/space#> .
@prefix vcard: <http://www.w3.org/2006/vcard/ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://dig.csail.mit.edu/2005/ajar/ajaw/data#Tabulator> doap:developer <https://www.w3.org/People/Berners-Lee/card#i> .

<http://dig.csail.mit.edu/2007/01/camp/data#course> :maker <https:/

In [49]:
d = {'col1': [1, 2], 'col2': [3, 4], 'col3': [3, 4]}
df = pd.DataFrame(data=d)
g = rdfpandas.to_graph(df)
s = g.serialize(format='turtle')


TypeError: 'int' object is not iterable

In [35]:
rdfpandas.graph.to_graph(df)

<Graph identifier=N3b9308ec698341349f7605c375b6b445 (<class 'rdflib.graph.Graph'>)>

In [45]:
df = df.replace(np.nan, '', regex=True)

In [47]:
df

,col1,col2
0,1,3
1,2,4
